In [ ]:
#### WELCOME TO WASP ####
### Please refer to the README for instructions ###

# Import Packages
import os
from tkinter import filedialog
import pandas as pd
import numpy as np
import math
import time

from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from combat.pycombat import pycombat
from umap import UMAP
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

import plotly.io as pio
pio.renderers.default = 'notebook'
import plotly.express as px
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
import seaborn as sns
from bioinfokit import analys, visuz


import warnings
warnings.filterwarnings("ignore")

# function to check for Total and % Missing
def nan_check(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent_1 = data.isnull().sum()/data.isnull().count()*100
    percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
    missing_data = pd.concat([total, percent_2], axis=1, keys=['Total Null', '% Missing'])
    return missing_data

# function to prepend a string to each list element
def prepend(list, str):
    str += '{0}'
    list = [str.format(i) for i in list]
    return(list)

# function used for removing nested lists
def reemovNestings(l):
    for i in l:
        if type(i) == list:
            reemovNestings(i)
        else:
            output.append(i)

# function for selecting number of PCs for 80% Variance explained
def find80p(variance):
    count = 0
    i = 0
    while count < 0.80:
        count += variance[i]
        i += 1
    return i

# Prompt to select WASP pipeline
WASP_pipeline = input('\n\nWelcome to WASP (Wyss Analysis Software for Proteomics).\n\nPlease choose a pipeline for Dimensionality Reduction and Differential Expression Analysis:\n\n0 ----- Single Cell\n1 ----- Bulk\n\n')
if WASP_pipeline == '0':
    print('\nWASP for Single Cell Proteomics')
else:
    print('\nWASP for Bulk Proteomics')

# naming
Experiment = input('\nEnter Experiment ID: ')
Plate = input('\nEnter Condition ID: ')
print('\nChoose Parent Directory')
parent_dir = filedialog.askdirectory()
Date = pd.to_datetime('today').date()
directory = Experiment
path = os.path.join(parent_dir, directory)
os.makedirs(path, exist_ok=True)

# input csv
print('Open PSM Excel File')
PSM = filedialog.askopenfilename()
UniProt = pd.read_excel('UniProt.xlsx')

# Load PSM as Pandas DataFrame
print('Reading PSM file (this make take a few minutes)')
df_raw = pd.read_excel(PSM)

'''# PSM QA
print('\nPSM Stats:')
df_raw.hist(column = 'DeltaM [ppm]')
print('\nAverage Delta M: ', round(df_raw['DeltaM [ppm]'].mean(), 2), 'ppm')
df_raw.hist(column = 'Deltam/z [Da]')
print('\nAverage Delta m/z : ', round(df_raw['Deltam/z [Da]'].mean(), 2), 'Da')
if df_raw['Checked'].nunique()==1:
    print('\nAll Checked == False? ', df_raw['Checked'].nunique()==1)
else:
    print(df_raw['Checked'].value_counts())
if df_raw['Confidence'].nunique()==1:
    print('\nAll Confidence == High? ', df_raw['Confidence'].nunique()==1)
else:
    print(df_raw['Confidence'].value_counts())
if df_raw['PSM Ambiguity'].nunique()==1:
    print('\nAll PSM Unambiguous? ', df_raw['PSM Ambiguity'].nunique()==1)
else:
    print('\nPSM Ambiguity: \n',df_raw['PSM Ambiguity'].value_counts())
df_raw.hist(column= 'Charge')
print('\nAverage Charge: ', round(df_raw['Charge'].mean(), 1))
print('\nAverage RT [min]: ', round(df_raw['RT [min]'].mean(), 1), 'min')
df_raw.hist(column = 'RT [min]')
df_raw.hist(column = 'Isolation Interference [%]')
df_raw.hist(column = 'Average Reporter S/N')'''

# Clean DataFrame
df = df_raw.rename(columns = {'Abundance: 126': '126', 'Abundance: 127N': '127N', 'Abundance: 127C': '127C',
               'Abundance: 128N': '128N', 'Abundance: 128C': '128C', 'Abundance: 129N': '129N',
              'Abundance: 129C': '129C', 'Abundance: 130N': '130N', 'Abundance: 130C': '130C',
              'Abundance: 131N': '131N', 'Abundance: 131C': '131C', 'Abundance: 132N': '132N',
              'Abundance: 132C': '132C', 'Abundance: 133N': '133N', 'Abundance: 133C': '133C',
              'Abundance: 134N': '134N', 'Abundance: 134C': '134C', 'Abundance: 135N': '135N'})
print('\nNumber PSM: ' + str(df.shape[0]))

# Remove Co-Isolated greater than 60%
print('\nRemoving 60% Co-Isolation')
df = df[~(df['Isolation Interference [%]'] >= 59.999)]
print('Number PSM: ' + str(df.shape[0]))

# Remove Keratin
print('Removing Keratin')
KRT = pd.read_excel('Keratin.xlsx')
KRT = KRT.astype(str)
KRT_list = '|'.join(list(KRT.Entry))
df = df.loc[~df['Master Protein Accessions'].str.contains(KRT_list, case=False, na=False)]
print('Number PSM: ' + str(df.shape[0]))

# Remove Contaminants
print('Removing Contaminants')
c2a = pd.read_csv('Contaminants.csv')
c2a = c2a.astype(str)
c2a = '|'.join(c2a['1'])
df = df.loc[~df['Master Protein Accessions'].str.contains(c2a, case=False, na=False)]

# Counts
print('\nCounts:')
print('Number PSM: ' + str(df.shape[0]))
print('Number of Peptides: ' + str(df['Annotated Sequence'].nunique()))
print('Number of Proteins: ' + str(df['Master Protein Accessions'].nunique()))
print('\n')

# Check for missing data by Channel
print('\nNull Table')
print(nan_check(df))

# Prompt to remove channels and drop from DataFrame
Bad_channels = input('\nRemove Channel(s)? \nEnter: y or n \n')
if Bad_channels == 'y':
    print('\nInput channels separated by a comma (,) NO SPACES')
    Remove = input('\nChannels to Remove: ')
    df = df.drop(Remove.split(','), axis=1)
else:
    pass

# Retrieve File ID Info and Remove Bad Injections
print('\nInjection Counts: \n')
abundance = df['File ID'].value_counts()
print(abundance)
Bad_injections = input('\nRemove Injection(s)? \nEnter: y or n \n')
if Bad_injections == 'y':
    print('\nInput File ID separated by a coma (,) NO SPACES')
    Inj_Remove = input('\nInjections to Remove: ')
    BI_list = '|'.join(list(Inj_Remove.split(',')))
    df = df.loc[~df['File ID'].str.contains(BI_list, case=False, na=False)]
else:
    pass

fid_count = df['File ID'].nunique()
print('\nUnique File ID: ' + str(fid_count))

# Identify Case and Control
print('\nInput channels separated by a comma (,) NO SPACES')
Case = input('\nEnter Case: ')
Control = input('Enter Control: ')

# Create Final DataFrame
df = df[['Annotated Sequence', 'Master Protein Accessions', 'File ID']].join(df[Control.split(',')].join(df[Case.split(',')]))
dfboxen = df[['Annotated Sequence', 'Master Protein Accessions', 'File ID']].join(df[Control.split(',')].join(df[Case.split(',')]))
#DEA_df = df.fillna(0)

# Plot Pre-Normalization
dfboxen = dfboxen.iloc[:,3:].apply(lambda x: np.log2(x), axis=0)

fig = sns.boxenplot(data=dfboxen, orient='h', width_method="linear")
plt.xlabel('Relative Abundance')
plt.ylabel('Channel')
plt.title('Pre-Normalization')
plt.savefig(f'{path}/{Experiment}_{Plate}_Pre-Normalization_{Date}.pdf', format="pdf", bbox_inches="tight")
plt.show(fig)

# Create Dictionaries for Groupby functions
Median = 'median'
Mean = 'mean'
Sum = 'sum'
Case_Median = {k:Median for k in Case.split(',')}
Control_Median = {k:Median for k in Control.split(',')}
Case_Mean = {k:Mean for k in Case.split(',')}
Control_Mean = {k:Mean for k in Control.split(',')}
DEA_agg = Control_Median|Case_Median
Case_Sum = {k:Sum for k in Case.split(',')}
Control_Sum = {k:Sum for k in Control.split(',')}
Heatmap_agg = Control_Median|Case_Median

### MOR (Median of Ratios) Normalization for Channel Normalization
# take the log of data
print('\nPerforming Median of Ratio Normalization for Channel Normalization')
log_data = np.log(df.iloc[:,3:])

# Take the average of each row
log_data['pseudo_reference'] = log_data.apply(lambda x: np.mean(x), axis=1)
log_data.reset_index(inplace=True)

# filter out proteins with infinity as average
filtered_log_data = log_data[log_data['pseudo_reference'] != float('-inf')]

# Subtract the protein pseudo-reference from log counts
ratio_data = filtered_log_data.iloc[:, 1:-1].apply(lambda x: x - filtered_log_data['pseudo_reference'], axis=0)

# Find the Median of the Ratios for each sample
sample_medians = ratio_data.median(axis=0)

# Convert medians to scaling factors
scaling_factors = np.exp(sample_medians)

# Divide original counts by scaling factors
manually_normalized = df.iloc[:,3:].div(scaling_factors, axis='columns')

# Plot Post-Normalization
fig2 = sns.boxenplot(data=manually_normalized.apply(lambda x: np.log2(x), axis=0),
                     orient='h', width_method="linear")
plt.xlabel('Relative Abundance')
plt.ylabel('Channel')
plt.title('Post-Normalization')
plt.savefig(f'{path}/{Experiment}_{Plate}_Post-Normalization_{Date}.pdf', format="pdf", bbox_inches="tight")
plt.show(fig2)

### this pipe leads to KNN 50% missing imputation
df = df.iloc[:,:3].join(manually_normalized)
df = df.reset_index(drop=True)

# break df and drop NaN
treated = df.filter(Case.split(','))
treated = df.iloc[:,:3].join(treated)
treated = treated.dropna(thresh=round(treated.shape[1]/2))
untreated = df.filter(Control.split(','))
untreated = df.iloc[:,:3].join(untreated)
untreated = untreated.dropna(thresh=round(untreated.shape[1]/2))

# preserve labels
labels = pd.DataFrame(treated.reset_index().iloc[:,:4])
columns = list(treated.iloc[:,3:].columns)
labels_untreated = pd.DataFrame(untreated.reset_index().iloc[:,:4])
columns_untreated = list(untreated.iloc[:,3:].columns)

# Make an instance and perform the imputation
print('\nImputing by Case/Control (this make take several minutes)')
start_time = time.time()
k = round(math.sqrt(treated.iloc[:,3:].shape[1]))
k2 = round(math.sqrt(untreated.iloc[:,3:].shape[1]))
imputer = KNNImputer(n_neighbors=k)
imputer2 = KNNImputer(n_neighbors=k2)
a = imputer.fit_transform(treated.iloc[:,3:])
a = labels.join(pd.DataFrame(a, columns = columns))
b = imputer2.fit_transform(untreated.iloc[:,3:])
b = labels_untreated.join(pd.DataFrame(b, columns = columns_untreated))
print('Time to impute: %s minutes' % round((time.time() - start_time)/60, 2))

# repiece and export
imputed = a.merge(b, how='inner').drop(columns=['index'])
# Apply log2 to each column
log2_imputed = imputed.iloc[:,3:].apply(lambda x: np.log2(x), axis=0)
log2_imputed = imputed.iloc[:,:3].join(log2_imputed)
log2_imputed = log2_imputed.dropna()
DEA_df = imputed.dropna()
imputed = imputed.dropna()
print(nan_check(imputed))
imputed.to_csv(f'{path}/{Experiment}_{Plate}_Imputed_{Date}.csv')

### SPLIT! for Single Cell/Bulk ###
if WASP_pipeline == '0':
    ### Remake Dataframe for Second Imputation
    imputed_lba = imputed.iloc[:,:3]

    # break df
    imputed_treated = imputed.filter(Case.split(','))
    imputed_treated = imputed_lba.join(imputed_treated)
    imputed_untreated = imputed.filter(Control.split(','))
    imputed_untreated = imputed_lba.join(imputed_untreated)
    
    # Group by File.ID and Master.Protein.Accessions columns and summarize columns 126 to 135N by median
    quant_treated = imputed_treated.groupby(['File ID', 'Master Protein Accessions']).agg(Case_Median).reset_index()
    quant_untreated = imputed_untreated.groupby(['File ID', 'Master Protein Accessions']).agg(Control_Median).reset_index()
    
    # Pivot longer to create a 'variable' column with column names
    quant_treated = pd.melt(quant_treated, id_vars=['File ID', 'Master Protein Accessions'], var_name='Channel', value_name='Abundance')
    quant_untreated = pd.melt(quant_untreated, id_vars=['File ID', 'Master Protein Accessions'], var_name='Channel', value_name='Abundance')

    # Create Sample ID
    quant_treated['Sample ID'] = quant_treated['Channel'].str.cat(quant_treated['File ID'], sep = '_')
    quant_treated = quant_treated.drop(['File ID', 'Channel'], axis=1)
    quant_untreated['Sample ID'] = quant_untreated['Channel'].str.cat(quant_untreated['File ID'], sep = '_')
    quant_untreated = quant_untreated.drop(['File ID', 'Channel'], axis=1)
    
    # Pivot wider to create new columns with variable names
    quant_treated = quant_treated.pivot(index='Master Protein Accessions', columns=['Sample ID'], values='Abundance')
    quant_untreated = quant_untreated.pivot(index='Master Protein Accessions', columns=['Sample ID'], values='Abundance')

    # Proteins with > 50% missing values
    thresh = round(quant_treated.shape[1]/2)
    na_quant_treated = quant_treated.reset_index().reset_index().dropna(axis=0, thresh=thresh)
    thresh = round(quant_untreated.shape[1]/2)
    na_quant_untreated = quant_untreated.reset_index().reset_index().dropna(axis=0, thresh=thresh)

    # preserve labels
    labels = pd.DataFrame(na_quant_treated.iloc[:,:2])
    columns = list(na_quant_treated.iloc[:,2:].columns)
    labels_untreated = pd.DataFrame(na_quant_untreated.iloc[:,:2])
    columns_untreated = list(na_quant_untreated.iloc[:,2:].columns)

    # Make an instance and perform the imputation
    k = round(math.sqrt(quant_treated.shape[1]))
    imputer = KNNImputer(n_neighbors=k)
    a = pd.DataFrame(imputer.fit_transform(na_quant_treated.iloc[:,2:]), columns = columns)
    a['index'] = list(labels['index'])
    a['Master Protein Accessions'] = list(labels['Master Protein Accessions'])
    cols = a.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    a = a[cols]
    k = round(math.sqrt(quant_untreated.shape[1]))
    imputer = KNNImputer(n_neighbors=k)
    b = pd.DataFrame(imputer.fit_transform(na_quant_untreated.iloc[:,2:]), columns = columns_untreated)
    b['index'] = list(labels_untreated['index'])
    b['Master Protein Accessions'] = list(labels_untreated['Master Protein Accessions'])
    cols = b.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    b = b[cols]

    # repiece
    imputed_2 = a.merge(b, how='inner').set_index('Master Protein Accessions').drop(columns=['index'])

    # make dataframe for PCA/UMAP
    pcDF = imputed_2.T
    mylist = Control.split(',')
    pcDF['Sample'] = pcDF.index
    pcDF[['Channel', 'File ID']] = pcDF.Sample.str.split("_", expand = True)
    pcDF['Treatment'] = np.where(pcDF['Channel'].isin(mylist), 'Control', 'Case')
    pcDF['batch'] = pcDF['File ID'].str.replace('F', '')
    pgp = pcDF.set_index('Sample')
    pgp.pop('Channel')
    pgp.pop('File ID')
    
    # Plot Pre-Combat Noramzalization
    plt.figure(figsize=(6,39))
    sns.boxenplot(data=pgp.iloc[:,:2].T, orient='h', width_method="linear")
    plt.xlabel('Relative Abundance')
    plt.ylabel('Channel')
    plt.title('Pre-Combat')
    plt.savefig(f'{path}/{Experiment}_{Plate}_Pre-Combat_{Date}.pdf', format="pdf", bbox_inches="tight")

    ### PyCombat Normalization 
    # make dataframes and labels
    print('\nPerforming PyCombat Batch Normalization for Injection Normalization')
    data_t = pgp.iloc[:,:-2].T
    data_l = pgp.Treatment
    batch = list(pgp.batch)

    # combat
    data_corrected = pycombat(data_t,batch)
    combat = data_corrected.T
    combat['Treatment'] = data_l
    combat.to_csv(f'{path}/{Experiment}_{Plate}_Combat_{Date}.csv')
    print('\nPost imputation/normalization number of proteins: ' + str(combat.shape[1]))
    
    # Plot Post-Combat
    plt.figure(figsize=(6,39))
    sns.boxenplot(data=combat.iloc[:,:2].T, orient='h', width_method="linear")
    plt.xlabel('Relative Abundance')
    plt.ylabel('Channel')
    plt.title('Post-Combat')
    plt.savefig(f'{path}/{Experiment}_{Plate}_Post-Combat_{Date}.pdf', format="pdf", bbox_inches="tight")
    
    # PCA viz
    print('\nPreparing Principal Component Analysis')
    df = combat
    X = combat.iloc[:,:-1]
    X.columns = X.columns.astype(str)
    y = combat.Treatment
    pca = PCA(n_components=min(X.shape))
    components = pca.fit_transform(X)

    fig = px.scatter(components, x=0, y=1, color=df['Treatment'], 
                     labels={
                         '0': 'PC 1',
                         '1': 'PC 2',
                         'color': Plate},
                    hover_name = df.index, title='Principal Component Analysis')
    fig.show()
    fig.write_image(f'{path}/{Experiment}_{Plate}_PCA_{Date}.png')
    fig.write_image(f'{path}/{Experiment}_{Plate}_PCA_{Date}.pdf')
    
    ### PCA nth component allows possibility to perform UMAP on PCs instead of normalized data
    # select pcs for 80% variance explained
    variance = pca.explained_variance_ratio_
    pcs = find80p(variance)
    npc = [x+1 for x in range(pcs)]
    PC_str = 'PC '

    # make PC_df Col labels
    PCA_cols = prepend(npc, PC_str)
    pca = PCA(n_components=pcs, svd_solver = 'auto')
    Principal_components=pca.fit_transform(X)
    pca_df = pd.DataFrame(data = Principal_components, columns = PCA_cols)
    pca_df['Treatment'] = list(y)

    # UMAP on principle components
    print('\nGenerating UMAP')
    features = X

    umap_2d = UMAP(n_components=2, n_neighbors=15, min_dist=0.75, n_epochs=5000, 
                   random_state=42)
    umap_3d = UMAP(n_components=3, n_neighbors=15, min_dist=0.75, n_epochs=5000, 
                   random_state=42)

    proj_2d = umap_2d.fit_transform(features)
    proj_3d = umap_3d.fit_transform(features)

    fig_2d = px.scatter(
        proj_2d, x=0, y=1,
        color=df.Treatment,
        labels={
            0: '',
            1: '',
            'color': Plate},
        hover_name=combat.index,
        title='Uniform Manifold Approximation & Projection'
    )
    fig_2d.update_traces(marker_size=10)

    fig_3d = px.scatter_3d(
        proj_3d, x=0, y=1, z=2,
        color=df.Treatment,
        labels={
            0: '',
            1: '',
            2: '',
            'color': Plate},
        hover_name=combat.index,
        title='Uniform Manifold Approximation & Projection'
    )
    fig_3d.update_traces(marker_size=5)

    fig_2d.show()
    fig_3d.show()

    fig_2d.write_image(f'{path}/{Experiment}_{Plate}_UMAP_2D_{Date}.png')
    fig_3d.write_image(f'{path}/{Experiment}_{Plate}_UMAP_3D_{Date}.png')
    fig_2d.write_image(f'{path}/{Experiment}_{Plate}_UMAP_2D_{Date}.pdf')
    fig_3d.write_image(f'{path}/{Experiment}_{Plate}_UMAP_3D_{Date}.pdf')

    ### Heatmap and Top 100 eigenvectors
    # get top 100 eigenvectors from PC1
    print('\nGetting Top 100 Eigenvectors')
    p = pd.DataFrame(abs( pca.components_ ))
    p.columns = combat.columns[0:-1]
    hm = p.T.sort_values(by=0, ascending=False).head(101)

    # Group by File.ID and Master.Protein.Accessions columns and summarize columns 126 to 135N by sum
    agg = DEA_df.groupby(['Master Protein Accessions']).agg(Heatmap_agg).reset_index()
    heatmap = agg.loc[agg['Master Protein Accessions'].isin(hm.index)].set_index('Master Protein Accessions')
    heatmap_csv = round(heatmap, 2)
    round(heatmap_csv, 1).to_excel(f'{path}/{Experiment}_{Plate}_Heatmap_Matrix_{Date}.xlsx')

    # Visualize Heatmap
    print('\nTop 40 Eigenvectors Heatmap')
    fig, ax = plt.subplots(figsize=(9, 9))
    sns.set(font_scale=0.9)
    heatmap_data = heatmap.head(40).apply(lambda x: np.log2(x), axis=0)
    hmll = np.where(heatmap.columns.isin(mylist), 'Control', 'Case')
    HMViz = sns.heatmap(heatmap_data, xticklabels=hmll, yticklabels=True, linewidth=.5)
    HMViz.set_xticklabels(HMViz.get_xticklabels(), rotation=360, ha='center', size='x-small')
    
    HMfig = HMViz.get_figure()
    HMfig.savefig(f'{path}/{Experiment}_{Plate}_Heatmap_{Date}.png')
    HMfig.savefig(f'{path}/{Experiment}_{Plate}_Heatmap_{Date}.pdf')

    ### Map Gene Symbol to Protein Accession
    # read file
    print('\nMapping Gene Symbol to Protein Accessions')
    ev = hm

    # rename Gene column and remove white space
    UniProt.rename(columns={'Gene Names': 'Gene Symbol', 'Entry': 'Master Protein Accessions', 'Protein names': 'Protein.Names'},
                   inplace=True)
    UniProt.drop(['Protein.Names'], axis = 1, inplace=True)
    UniProt['Gene Symbol'] = UniProt['Gene Symbol'].str.split(' ').str[0]

    # make dataframe
    ev = ev.reset_index()
    ev['Gene Symbol'] = ''
    cllist = [x for x in range(len(ev.columns)-2)]
    output = []
    nested_cols = ['Master Protein Accessions', 'Gene Symbol', cllist]
    reemovNestings(nested_cols)
    cols = output
    ev = ev[output]
    ev['Master Protein Accessions'] = ev['Master Protein Accessions'].str.split('; ').str[0]

    # merge
    mapped = pd.merge(UniProt, ev, on = 'Master Protein Accessions', how = 'right')

    # clean
    mapped.drop(['Gene Symbol_y'], axis=1, inplace=True)
    mapped.rename(columns={'Gene Symbol_x': 'Gene Symbol'},inplace=True)
    mapped.rename(columns=dict(zip(cllist, PCA_cols)), inplace=True)

    # find unmapped Genes
    print('\n' + str(mapped['Gene Symbol'].isna().sum()) + ' IDs were not mapped')
    mapped.replace(np.NaN, 'NaN', inplace=True)
    for i in range(mapped.shape[0]):
        if mapped['Gene Symbol'].iloc[i] == 'NaN':
            print(mapped['Master Protein Accessions'].iloc[i])
        else:
            pass

    # export
    round(mapped, 3).to_excel(f'{path}/{Experiment}_{Plate}_100Eigenvectors_{Date}.xlsx')

    ### Differential Expression Analysis
    # Prepare DF
    #DEA_df = DEA_imputed
    DEA_df = DEA_df.groupby(['Master Protein Accessions']).agg(DEA_agg).reset_index()
    counts = DEA_df.set_index('Master Protein Accessions').T.astype('int')
    clinical = DEA_df.set_index('Master Protein Accessions').T
    clinical = pd.DataFrame(clinical.index)
    clinical['Treatment'] = np.where(clinical[0].isin(mylist), 'Control', 'Treatment')
    clinical = clinical.set_index(0)

    # Build Model
    dds = DeseqDataSet(
        counts=counts,
        clinical=clinical,
        design_factors='Treatment',
        refit_cooks=True,
        n_cpus=8)

    # Run DESeq2
    print('\nRunning DESeq2')
    dds.deseq2()

    stat_res = DeseqStats(dds, n_cpus=8)
    stats = round(pd.DataFrame(stat_res.summary()), 2)

    round(stat_res.results_df, 3).to_excel(f'{path}/{Experiment}_{Plate}_Log2FC_WaldP-value_Proteins_{Date}.xlsx')

    ### Map Gene Symbol to Protein Accession
    # read file
    print('\nMapping Gene Symbol to Protein Accessions')
    RegulatedProteins = stat_res.results_df
    RegulatedProteins.rename(columns={'baseMean': 'Base Mean', 'log2FoldChange': 'log2 Fold Change', 'pvalue': 'P-value'},
                            inplace=True)
    RegulatedProteins.drop(['lfcSE', 'stat', 'padj'], axis=1, inplace=True)
    RegulatedProteins['Gene Symbol'] = ''
    RegulatedProteins['Fold Change'] = 2**RegulatedProteins['log2 Fold Change']
    RP = ['Gene Symbol', 'Base Mean', 'log2 Fold Change', 'Fold Change', 'P-value']
    RegulatedProteins = RegulatedProteins[RP]
    RegulatedProteins.reset_index(inplace=True)
    RegulatedProteins['Master Protein Accessions'] = RegulatedProteins['Master Protein Accessions'].str.split('; ').str[0]

    # merge
    mapped = pd.merge(UniProt, RegulatedProteins, on = 'Master Protein Accessions', how = 'right')

    # clean
    mapped.drop(['Gene Symbol_y'], axis=1, inplace=True)
    mapped.rename(columns={'Gene Symbol_x': 'Gene Symbol'},inplace=True)
    mapped.rename(columns=dict(zip(cllist, PCA_cols)), inplace=True)
    
    # Volcano Plot with Protein Names
    try:
        visuz.GeneExpression.volcano(df=mapped.dropna(), lfc='log2 Fold Change', pv='P-value',
                                     lfc_thr=(0.58, 0.58), geneid='Master Protein Accessions',
                                     genenames='deg', gfont=4, color= ('red', 'grey', 'blue'), sign_line=True)
    except AssertionError:
        print('\nNo Significantly, Differentially Abundant Proteins')
        pass

    # find unmapped Genes
    print('\n' + str(mapped['Gene Symbol'].isna().sum()) + ' IDs were not mapped')
    mapped.replace(np.NaN, 'NaN', inplace=True)
    for i in range(mapped.shape[0]):
        if mapped['Gene Symbol'].iloc[i] == 'NaN':
            print(mapped['Master Protein Accessions'].iloc[i])
        else:
            pass

    # export
    round(mapped, 3).to_excel(f'{path}/{Experiment}_{Plate}_log2FC_WaldP-value_Genes_{Date}.xlsx')

    print('\n\nProgram Complete')

else:
    ### BULK
    # make dataframe for PCA/UMAP
    pcDF = DEA_df.iloc[:,3:].T
    mylist = Control.split(',')
    pcDF['Sample'] = pcDF.index
    pcDF['Treatment'] = np.where(pcDF['Sample'].isin(mylist), 'Control', 'Case')
    pgp = pcDF.set_index('Sample')

    # PCA
    print('\nPreparing Principal Component Analysis')
    df = pgp
    X = pgp.iloc[:,:-3]
    X.columns = X.columns.astype(str)
    y = pgp.Treatment
    pca = PCA(n_components=min(X.shape))
    components = pca.fit_transform(X)

    fig = px.scatter(components, x=0, y=1, color=df['Treatment'],
                     labels={
                         0: 'PC 1',
                         1: 'PC 2',
                         'color': Plate},
                     text=pgp.index, title='Principal Component Analysis')
    fig.update_traces(textposition='top center')
    fig.update_traces(marker_size=10)
    fig.show()
    fig.write_image(f'{path}/{Experiment}_{Plate}_Bulk_PCA_{Date}.png')
    fig.write_image(f'{path}/{Experiment}_{Plate}_Bulk_PCA_{Date}.pdf')

    ### PCA nth component allows possibility to perform UMAP on PCs instead of normalized data 
    # select pcs for 80% variance explained
    variance = pca.explained_variance_ratio_
    pcs = find80p(variance)
    npc = [x+1 for x in range(pcs)]
    PC_str = 'PC '

    # make PC_df Col labels
    PCA_cols = prepend(npc, PC_str)
    pca = PCA(n_components=pcs, svd_solver = 'auto')
    Principal_components=pca.fit_transform(X)
    pca_df = pd.DataFrame(data = Principal_components, columns = PCA_cols)
    pca_df['Treatment'] = list(y)

    # UMAP on principle components
    print('\nGenerating UMAP')
    features = X

    umap_2d = UMAP(n_components=2, n_neighbors=len(Case.split(',')), n_epochs=5000, 
                   random_state=42)
    umap_3d = UMAP(n_components=3, n_neighbors=len(Case.split(',')), n_epochs=5000, 
                   random_state=42)

    proj_2d = umap_2d.fit_transform(features)
    proj_3d = umap_3d.fit_transform(features)

    fig_2d = px.scatter(
        proj_2d, x=0, y=1,
        color=df.Treatment,
        labels={
            0: '',
            1: '',
            'color': Plate},
        hover_name=df.index,
        text=pgp.index,
        title='Uniform Manifold Approximation & Projection'
    )
    fig_2d.update_traces(textposition='top center')
    fig_2d.update_traces(marker_size=10)

    fig_3d = px.scatter_3d(
        proj_3d, x=0, y=1, z=2,
        color=df.Treatment,
        labels={
            0: '',
            1: '',
            2: '',
            'color': Plate},
        text=df.index,
        title='Uniform Manifold Approximation & Projection'
    )
    fig_3d.update_traces(textposition='top center')
    fig_3d.update_traces(marker_size=5)

    fig_2d.show()
    fig_3d.show()

    fig_2d.write_image(f'{path}/{Experiment}_{Plate}_Bulk_UMAP_2D_{Date}.png')
    fig_3d.write_image(f'{path}/{Experiment}_{Plate}_Bulk_UMAP_3D_{Date}.png')
    fig_2d.write_image(f'{path}/{Experiment}_{Plate}_Bulk_UMAP_2D_{Date}.pdf')
    fig_3d.write_image(f'{path}/{Experiment}_{Plate}_Bulk_UMAP_3D_{Date}.pdf')
    
    ### Differential Expression Analysis
    # Prepare DF
    #DEA_df = DEA_imputed
    DEA_df = DEA_df.groupby(['Master Protein Accessions']).agg(DEA_agg).reset_index()
    counts = DEA_df.set_index('Master Protein Accessions').T.astype('int')
    clinical = DEA_df.set_index('Master Protein Accessions').T
    clinical = pd.DataFrame(clinical.index)
    clinical['Treatment'] = np.where(clinical[0].isin(mylist), 'Control', 'Treatment')
    clinical = clinical.set_index(0)

    # Build Model
    dds = DeseqDataSet(
        counts=counts,
        clinical=clinical,
        design_factors='Treatment',
        refit_cooks=True,
        n_cpus=8)

    # Run DESeq2
    print('\nRunning DESeq2')
    dds.deseq2()

    stat_res = DeseqStats(dds, n_cpus=8)
    stats = round(pd.DataFrame(stat_res.summary()), 2)

    round(stat_res.results_df, 3).to_excel(f'{path}/{Experiment}_{Plate}_Bulk_Log2FC_WaldP-value_Proteins_{Date}.xlsx')

    ### Map Gene Symbol to Protein Accession
    # read file
    print('\nMapping Gene Symbol to Protein Accessions')
    RegulatedProteins = stat_res.results_df
    RegulatedProteins.rename(columns={'baseMean': 'Base Mean', 'log2FoldChange': 'log2 Fold Change', 'pvalue': 'P-value'},
                            inplace=True)
    RegulatedProteins.drop(['lfcSE', 'stat', 'padj'], axis=1, inplace=True)
    RegulatedProteins['Gene Symbol'] = ''
    RegulatedProteins['Fold Change'] = 2**RegulatedProteins['log2 Fold Change']
    RP = ['Gene Symbol', 'Base Mean', 'log2 Fold Change', 'Fold Change', 'P-value']
    RegulatedProteins = RegulatedProteins[RP]
    RegulatedProteins.reset_index(inplace=True)
    RegulatedProteins['Master Protein Accessions'] = RegulatedProteins['Master Protein Accessions'].str.split('; ').str[0]

    # rename Gene column and remove white space
    UniProt.rename(columns={'Gene Names': 'Gene Symbol', 'Entry': 'Master Protein Accessions', 'Protein names': 'Protein.Names'},
                   inplace=True)
    UniProt.drop(['Protein.Names'], axis = 1, inplace=True)
    UniProt['Gene Symbol'] = UniProt['Gene Symbol'].str.split(' ').str[0]

    # merge
    mapped = pd.merge(UniProt, RegulatedProteins, on = 'Master Protein Accessions', how = 'right')

    # clean
    mapped.drop(['Gene Symbol_y'], axis=1, inplace=True)
    mapped.rename(columns={'Gene Symbol_x': 'Gene Symbol'},inplace=True)

    # Volcano Plot with Protein Names
    try:
        visuz.GeneExpression.volcano(df=mapped.dropna(), lfc='log2 Fold Change', pv='P-value',
                                     lfc_thr=(0.58, 0.58), geneid='Master Protein Accessions',
                                     genenames='deg', gfont=4, color= ('red', 'grey', 'blue'), sign_line=True)
    except AssertionError:
        print('\nNo Significantly, Differentially Abundant Proteins')
        pass
                         
    # find unmapped Genes
    print('\n' + str(mapped['Gene Symbol'].isna().sum()) + ' IDs were not mapped')
    mapped.replace(np.NaN, 'NaN', inplace=True)
    for i in range(mapped.shape[0]):
        if mapped['Gene Symbol'].iloc[i] == 'NaN':
            print(mapped['Master Protein Accessions'].iloc[i])
        else:
            pass

    # export
    round(mapped, 3).to_excel(f'{path}/{Experiment}_{Plate}_Bulk_log2FC_WaldP-value_Genes_{Date}.xlsx')

    print('\n\nProgram Complete')  